**The Battle of Neighborhoods: Finding the Best Location for an African/Caribbean Store and Restaurant in Toronto, CA**

**Introduction:**

The purpose of this project is to find the best location or neighborhood for an African/ Caribbean store or restaurant. In recent times, immigration of Africans and Caribbean to Canada especially the Toronto region has increased significantly. This project will interest entrepreneur trying to setup Afro-Caribbean store and restaurant to capitalize on this increase in population while maximizing sales and profit by situating the store or restaurant in a less competitive neighborhood.
The idea is to categorize the neighborhood into clusters and apply the Foursquare API to find the neighborhood with the least number of Afro-Caribbean stores or restaurants. An efficient and easy way will be to use Foursquare explore tool to obtain a list of existing stores in our clusters and then recommend a distant location to capture others and newer market that might be too far from the existing locations. Another method will be to use the demography of Toronto and try to find where most Africans and Caribbean dwell, go to school and work. However, the later method is a mere common sense without any Data insight and extensively manual.

**Data:**

As mentioned earlier we will be using the Foursquare API data for different neighborhood in Toronto. Foursquare is a location data provider with information about all manner of venues and events within an area of interest. Such information includes venue names, locations, menus and even photos as well as Free Wi-Fi locations. For each neighborhood selected, we will use a radius of 500 meter and a limit of 100 locations. The data retrieved from Foursquare contained information of venues within a specified distance of the longitude and latitude of the postcodes. The information obtained per venue as follows:
Neighborhood, neighborhood latitude, neighborhood longitude
Venue, name of the venue, venue latitude, venue longitude, venue category
For the Neighborhood data will be scrapping the data from the Canadian neighborhood Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  
Some of the Python libraries that will be used in the Data wrangling include:
a.	Pandas: For creating dataframe
b.	Scikit Learn: For importing k-means clustering
c.	Matplotlib: Python Plotting Module
d.	Folium: Python visualization library would be used to visualize the neighborhoods cluster distribution of using interactive leaflet map
e.	JSON: Library to handle JSON files
f.	Geocoder: To retrieve Location Data


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
!pip install folium

     |████████████████████████████████| 93 kB 2.9 MB/s  eta 0:00:01


In [4]:
import folium # map rendering library

**Scrapping Data from Wikipedia Page**

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = dfs[0]
df1 = df[['Postal Code', 'Borough', 'Neighbourhood']]
df1.head(100)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


**Group Neighbourhoods by Postal Code and Bursough

In [6]:
df2 = df1.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
df2.head(20)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


**Obtain Latitude and Longitude from CSV**

In [7]:
url2 = "https://cocl.us/Geospatial_data"
dfgeo = pd.read_csv(url2)

dfgeo.head(20)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


**Merge dfgeo and df2 dataframes**

In [8]:
df_Final = pd.merge(df2, dfgeo, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
# remove repeated "Postal Code" column
df_Final.drop("Postal Code", axis=1, inplace=True)
df_Final.head(20)

,Borough,Neighbourhood,Latitude,Longitude
0,Not assigned,Not assigned,NaN,NaN
1,Scarborough,"Malvern, Rouge",43.806686,-79.194353
2,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
3,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
4,Scarborough,Woburn,43.770992,-79.216917
5,Scarborough,Cedarbrae,43.773136,-79.239476
6,Scarborough,Scarborough Village,43.744734,-79.239476
7,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
8,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
9,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476


In [9]:
#Cleaning Data fram and droping Not Assigned
df_Final = df_Final.dropna()
df_Final.head(103
             )

,Borough,Neighbourhood,Latitude,Longitude
1,Scarborough,"Malvern, Rouge",43.806686,-79.194353
2,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
3,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
4,Scarborough,Woburn,43.770992,-79.216917
5,Scarborough,Cedarbrae,43.773136,-79.239476
6,Scarborough,Scarborough Village,43.744734,-79.239476
7,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
8,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
9,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
10,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [10]:
df_Final.shape

(103, 4)

**Exploring and clustering the neighborhoods in Toronto**

***Fisrt Get Latitude and Longitude for Toronto***

In [11]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


***Creating a map of Toronto with neighbourhoods superimposed on top.***

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_Final['Latitude'], df_Final['Longitude'], df_Final['Borough'], df_Final['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Exploring All Neighbourhoods in our data frame df_Final**

In [13]:
# The code was removed by Watson Studio for sharing.

In [14]:
df_Final.loc[70, 'Neighbourhood']

'Lawrence Park'

In [15]:
neighbourhood_latitude = df_Final.loc[70, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df_Final.loc[70, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df_Final.loc[70, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


In [16]:
LIMIT = 200 # limit of number of venues returned by Foursquare API



radius = 2000 # define radius of coordinates

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RXDNHV1DTYMRSVNPKMY13PWS4ON5JZELUOAPYYROENS321H1&client_secret=P0E4LJGT2UVRVX30SZETNHLIUY0F4J4ODSC3X15P0ZJNK32X&v=20180605&ll=43.7280205,-79.3887901&radius=2000&limit=200'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fcd28a87c5f250923a79223'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 93,
  'suggestedBounds': {'ne': {'lat': 43.746020518000016,
    'lng': -79.36392754210921},
   'sw': {'lat': 43.71002048199998, 'lng': -79.41365265789078}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledLatLngs': [{'label':

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(200)

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Sherwood Park,Park,43.716551,-79.387776
2,Granite Club,Gym / Fitness Center,43.733043,-79.381986
3,Sheridan Nurseries,Flower Shop,43.719005,-79.400500
4,Alexander Muir Memorial Gardens,Garden,43.721315,-79.400822
5,T-buds,Tea Room,43.731247,-79.403640
6,Bobbette & Belle,Bakery,43.731339,-79.403769
7,For The Win Cafe,Bubble Tea Shop,43.728636,-79.403255
8,The Belly Buster Submarines,Sandwich Place,43.733743,-79.404390
9,Shinobu by Maki Sushi,Japanese Restaurant,43.732562,-79.404147


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

93 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
Toronto_venues = getNearbyVenues(names=df_Final['Neighbourhood'],
                                   latitudes=df_Final['Latitude'],
                                   longitudes=df_Final['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [27]:
print(Toronto_venues.shape)
Toronto_venues.head(200)

(8589, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
1,"Malvern, Rouge",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
2,"Malvern, Rouge",43.806686,-79.194353,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports
3,"Malvern, Rouge",43.806686,-79.194353,Toronto Zoo,43.820582,-79.181551,Zoo
4,"Malvern, Rouge",43.806686,-79.194353,Polar Bear Exhibit,43.823372,-79.185145,Zoo
5,"Malvern, Rouge",43.806686,-79.194353,Orangutan Exhibit,43.818413,-79.182548,Zoo Exhibit
6,"Malvern, Rouge",43.806686,-79.194353,Penguin Exhibit,43.819435,-79.185959,Zoo Exhibit
7,"Malvern, Rouge",43.806686,-79.194353,Harvey's,43.800020,-79.198307,Restaurant
8,"Malvern, Rouge",43.806686,-79.194353,Gorilla Exhibit,43.819080,-79.184235,Zoo Exhibit
9,"Malvern, Rouge",43.806686,-79.194353,Australasia Pavillion,43.822563,-79.183286,Zoo Exhibit


In [157]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,100,100,100,100,100,100
"Alderwood, Long Branch",100,100,100,100,100,100
"Bathurst Manor, Wilson Heights, Downsview North",51,51,51,51,51,51
Bayview Village,54,54,54,54,54,54
"Bedford Park, Lawrence Manor East",98,98,98,98,98,98
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",44,44,44,44,44,44
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100


In [28]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 329 uniques categories.


**Analyze Each Neighborhood in toronto**

In [29]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American R

**Grouping rows by neighborhood and by taking the sum of occurrence of each category**

In [30]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').sum().reset_index()
Toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American R

**Putting into a pandas dataframe**


First, let's write a function to sort the venues in descending order.

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(100)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Coffee Shop,Restaurant,Pharmacy,Indian Restaurant,Sandwich Place,Bank,Breakfast Spot,Shopping Mall,Cantonese Restaurant
1,"Alderwood, Long Branch",Coffee Shop,Department Store,Fast Food Restaurant,Restaurant,Burger Joint,Electronics Store,Grocery Store,Breakfast Spot,Clothing Store,Pizza Place
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Park,Pizza Place,Restaurant,Bank,Sandwich Place,Gas Station,Convenience Store,Deli / Bodega,Pharmacy
3,Bayview Village,Park,Chinese Restaurant,Coffee Shop,Intersection,Bank,Gas Station,Clothing Store,Grocery Store,Café,Pharmacy
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sushi Restaurant,Italian Restaurant,Bakery,Restaurant,Sandwich Place,Bagel Shop,Café,Pizza Place,Pub
5,Berczy Park,Coffee Shop,Café,Hotel,Restaurant,Park,Japanese Restaurant,Farmers Market,Beer Bar,Department Store,Italian Restaurant
6,"Birch Cliff, Cliffside West",Coffee Shop,Grocery Store,Pizza Place,Gym,Beer Store,Gas Station,Bank,Park,Restaurant,Supermarket
7,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Bakery,Restaurant,Park,Café,Gift Shop,Italian Restaurant,Asian Restaurant,Bar,Seafood Restaurant
8,"Business reply mail Processing Centre, South C...",Coffee Shop,Brewery,Park,Italian Restaurant,Bakery,Café,Beach,Bar,Indian Restaurant,American Restaurant
9,"CN Tower, King and Spadina, Railway Lands, Har...",Park,Coffee Shop,Café,Scenic Lookout,Italian Restaurant,Hotel,Gym,Harbor / Marina,Restaurant,Plaza


In [163]:
Toronto_grouped.columns.values.tolist()

['Neighbourhood',
 'Accessories Store',
 'Afghan Restaurant',
 'African Restaurant',
 'Airport',
 'American Restaurant',
 'Amphitheater',
 'Antique Shop',
 'Aquarium',
 'Arcade',
 'Argentinian Restaurant',
 'Art Gallery',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'Auto Dealership',
 'Auto Garage',
 'Automotive Shop',
 'BBQ Joint',
 'Baby Store',
 'Badminton Court',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Baseball Stadium',
 'Basketball Court',
 'Basketball Stadium',
 'Bath House',
 'Beach',
 'Beach Bar',
 'Beer Bar',
 'Beer Store',
 'Big Box Store',
 'Bike Shop',
 'Bistro',
 'Board Shop',
 'Bookstore',
 'Botanical Garden',
 'Boutique',
 'Bowling Alley',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Bridal Shop',
 'Bridge',
 'Bubble Tea Shop',
 'Burger Joint',
 'Burrito Place',
 'Bus Line',
 'Bus Station',
 'Bus Stop',
 'Butcher',
 'Café',
 'Cajun / Creole Restaurant',
 'Candy Store',
 'Cantonese Restaurant',
 'Caribbean Res

**Extracting Restaurants in the Greater Toronto Area with with African,Caribbean, Egyptian, Meditarrenean, Ethiopian in the Names**

In [32]:
Sum_Afro_restaurants = Toronto_grouped['African Restaurant']+Toronto_grouped['Caribbean Restaurant']+Toronto_grouped['Egyptian Restaurant']+Toronto_grouped['Ethiopian Restaurant']

In [33]:
Toronto_Afro_restaurants = pd.DataFrame(data={'Neighbourhood': Toronto_grouped['Neighbourhood'],'Sum_Afro_restaurants':Sum_Afro_restaurants,'African Restaurant':Toronto_grouped['African Restaurant'],
                                  'Caribbean Restaurant':Toronto_grouped['Caribbean Restaurant'],'Egyptian Restaurant':Toronto_grouped['Egyptian Restaurant'],'Ethiopian Restaurant':Toronto_grouped['Ethiopian Restaurant']})
Toronto_Afro_restaurants.head(100)

,Neighbourhood,Sum_Afro_restaurants,African Restaurant,Caribbean Restaurant,Egyptian Restaurant,Ethiopian Restaurant
0,Agincourt,2,0,2,0,0
1,"Alderwood, Long Branch",0,0,0,0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0
3,Bayview Village,0,0,0,0,0
4,"Bedford Park, Lawrence Manor East",0,0,0,0,0
5,Berczy Park,0,0,0,0,0
6,"Birch Cliff, Cliffside West",0,0,0,0,0
7,"Brockton, Parkdale Village, Exhibition Place",1,0,0,0,1
8,"Business reply mail Processing Centre, South C...",2,0,1,1,0
9,"CN Tower, King and Spadina, Railway Lands, Har...",2,0,2,0,0


**Combining the Two data frames to Add burrows as well as coordinates on the same data frame in preparation for clustering**

In [34]:
df_final1 = pd.merge(df_Final,Toronto_Afro_restaurants,on='Neighbourhood')
df_final1.head()

,Borough,Neighbourhood,Latitude,Longitude,Sum_Afro_restaurants,African Restaurant,Caribbean Restaurant,Egyptian Restaurant,Ethiopian Restaurant
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,1,1,0,0
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,0,0,0,0
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0
3,Scarborough,Woburn,43.770992,-79.216917,0,0,0,0,0
4,Scarborough,Cedarbrae,43.773136,-79.239476,1,0,1,0,0


In [35]:
Toronto_Restaurant_clustering = df_final1.drop('Neighbourhood', 1)

In [46]:
Toronto_Restaurant_clustering = df_final1.drop('Borough', 1)
Toronto_Restaurant_clustering.head(10)

,Cluster Labels,Neighbourhood,Latitude,Longitude,Sum_Afro_restaurants,African Restaurant,Caribbean Restaurant,Egyptian Restaurant,Ethiopian Restaurant
0,2,"Malvern, Rouge",43.806686,-79.194353,2,1,1,0,0
1,0,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,0,0,0,0
2,0,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0
3,0,Woburn,43.770992,-79.216917,0,0,0,0,0
4,2,Cedarbrae,43.773136,-79.239476,1,0,1,0,0
5,2,Scarborough Village,43.744734,-79.239476,1,0,1,0,0
6,0,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,0,0,0,0,0
7,0,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,0,0,0,0,0
8,0,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0,0,0,0,0
9,0,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,0,0,0,0


In [37]:
Toronto_Restaurant_clustering = df_final1.drop(['Neighbourhood','Borough'], axis=1)
Toronto_Restaurant_clustering.head()

,Latitude,Longitude,Sum_Afro_restaurants,African Restaurant,Caribbean Restaurant,Egyptian Restaurant,Ethiopian Restaurant
0,43.806686,-79.194353,2,1,1,0,0
1,43.784535,-79.160497,0,0,0,0,0
2,43.763573,-79.188711,0,0,0,0,0
3,43.770992,-79.216917,0,0,0,0,0
4,43.773136,-79.239476,1,0,1,0,0


**CLUSTERING the Neigbourhoods**

In [38]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_Restaurant_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 0, 0, 2, 2, 0, 0, 0, 0], dtype=int32)

***creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood***

In [40]:
# add clustering labels


df_final1.insert(0, 'Cluster Labels', kmeans.labels_)

df_final1.head() # check the last columns!

,Cluster Labels,Borough,Neighbourhood,Latitude,Longitude,Sum_Afro_restaurants,African Restaurant,Caribbean Restaurant,Egyptian Restaurant,Ethiopian Restaurant
0,2,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,1,1,0,0
1,0,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,0,0,0,0
2,0,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0
3,0,Scarborough,Woburn,43.770992,-79.216917,0,0,0,0,0
4,2,Scarborough,Cedarbrae,43.773136,-79.239476,1,0,1,0,0


 **Visualizing the resulting clusters**

In [41]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_final1['Latitude'], df_final1['Longitude'],df_final1['Neighbourhood'],df_final1['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Examining Clusters**

**Cluster 1: No African/Caribbean Related Restaurant represented by Red Circles on the Map**

In [43]:
df_final1.loc[df_final1['Cluster Labels'] == 0, df_final1.columns[[1] + list(range(2, df_final1.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Sum_Afro_restaurants,African Restaurant,Caribbean Restaurant,Egyptian Restaurant,Ethiopian Restaurant
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,0,0,0,0
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0
3,Scarborough,Woburn,43.770992,-79.216917,0,0,0,0,0
6,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,0,0,0,0,0
7,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,0,0,0,0,0
8,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0,0,0,0,0
9,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,0,0,0,0
11,Scarborough,"Wexford, Maryvale",43.750072,-79.295849,0,0,0,0,0
16,Scarborough,Upper Rouge,43.836125,-79.205636,0,0,0,0,0
17,North York,Hillcrest Village,43.803762,-79.363452,0,0,0,0,0


**Cluster 2: Two to Three African/Caribbean related Restaurants represented by purple circle on the Map**

In [45]:
df_final1.loc[df_final1['Cluster Labels'] == 1, df_final1.columns[[1] + list(range(2, df_final1.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Sum_Afro_restaurants,African Restaurant,Caribbean Restaurant,Egyptian Restaurant,Ethiopian Restaurant
12,Scarborough,Agincourt,43.794200,-79.262029,2,0,2,0,0
13,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",43.781638,-79.304302,2,0,2,0,0
15,Scarborough,"Steeles West, L'Amoreaux West",43.799525,-79.318389,2,0,2,0,0
18,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,2,0,2,0,0
25,North York,Parkwoods,43.753259,-79.329656,2,0,2,0,0
37,East Toronto,The Beaches,43.676357,-79.293031,2,0,2,0,0
41,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,0,2,0,0
66,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,2,0,2,0,0
67,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,3,0,3,0,0
68,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,2,0,2,0,0


**Cluster 3: Two to One African/Caribbean related Restaurants, represented by Blue circles on the Map**

In [214]:
df_final1.loc[df_final1['Cluster Labels'] == 2, df_final1.columns[[1] + list(range(2, df_final1.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Sum_Afro_restaurants,African Restaurant,Caribbean Restaurant,Egyptian Restaurant,Ethiopian Restaurant
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,1,1,0,0
4,Scarborough,Cedarbrae,43.773136,-79.239476,1,0,1,0,0
5,Scarborough,Scarborough Village,43.744734,-79.239476,1,0,1,0,0
10,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304,1,0,1,0,0
14,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577,1,0,1,0,0
26,North York,Don Mills,43.745906,-79.352188,1,0,1,0,0
27,North York,Don Mills,43.725900,-79.340923,1,0,1,0,0
29,North York,"Northwood Park, York University",43.767980,-79.487262,1,0,1,0,0
43,East Toronto,Studio District,43.659526,-79.340923,2,0,1,1,0
50,Downtown Toronto,Rosedale,43.679563,-79.377529,1,0,1,0,0


**Cluster 4: Mostly East African/Egyptian Restaurants, represented By Green circles on the Map**

In [216]:
df_final1.loc[df_final1['Cluster Labels'] == 3
                   , df_final1.columns[[1] + list(range(2, df_final1.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Sum_Afro_restaurants,African Restaurant,Caribbean Restaurant,Egyptian Restaurant,Ethiopian Restaurant
42,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,0,0,1,1
75,Downtown Toronto,Christie,43.669542,-79.422564,1,0,0,0,1
76,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,2,0,0,0,2
78,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,1,0,0,0,1


**Cluster 5: HighestNumber of African/Caribbean related Restaurants respresnted by Orange Circle on the Map**

In [218]:
df_final1.loc[df_final1['Cluster Labels'] == 4, df_final1.columns[[1] + list(range(2, df_final1.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Sum_Afro_restaurants,African Restaurant,Caribbean Restaurant,Egyptian Restaurant,Ethiopian Restaurant
36,East York,Woodbine Heights,43.695344,-79.318389,4,0,1,0,3
40,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,6,0,2,1,3
